In [3]:
import os
import math
from numpy import * 
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')

### 若要測試單一個，請以以下方式引入單一資料集
<span id="singleRunHead"></span>

In [1]:
# dir_data = '../datasets/'
# DATASET_CSV_NAME = '1_zoo.csv'
# DATASET_NAME = 'zoo'
# dir_train_test_data = f'../train_test_datasets/{DATASET_NAME}'
# train_data_loc = os.path.join(dir_data, DATASET_CSV_NAME)
# print('Path of read in data: %s' % (train_data_loc))
# data = pd.read_csv(train_data_loc)
# data.head()

In [2]:
# len(data.columns)

In [22]:
# train_1_data_loc = os.path.join(dir_train_test_data, 'train_1.csv')
# train_1 = pd.read_csv(train_1_data_loc)
# train_2_data_loc = os.path.join(dir_train_test_data, 'train_2.csv')
# train_2 = pd.read_csv(train_2_data_loc)
# train_3_data_loc = os.path.join(dir_train_test_data, 'train_3.csv')
# train_3 = pd.read_csv(train_3_data_loc)
# train_4_data_loc = os.path.join(dir_train_test_data, 'train_4.csv')
# train_4 = pd.read_csv(train_4_data_loc)
# train_5_data_loc = os.path.join(dir_train_test_data, 'train_5.csv')
# train_5 = pd.read_csv(train_5_data_loc)
# test_1_data_loc = os.path.join(dir_train_test_data, 'test_1.csv')
# test_1 = pd.read_csv(test_1_data_loc)
# test_2_data_loc = os.path.join(dir_train_test_data, 'test_2.csv')
# test_2 = pd.read_csv(test_2_data_loc)
# test_3_data_loc = os.path.join(dir_train_test_data, 'test_3.csv')
# test_3 = pd.read_csv(test_3_data_loc)
# test_4_data_loc = os.path.join(dir_train_test_data, 'test_4.csv')
# test_4 = pd.read_csv(test_4_data_loc)
# test_5_data_loc = os.path.join(dir_train_test_data, 'test_5.csv')
# test_5 = pd.read_csv(test_5_data_loc)
# del train_1['numbers']
# del train_2['numbers']
# del train_3['numbers']
# del train_4['numbers']
# del train_5['numbers']
# del test_1['numbers']
# del test_2['numbers']
# del test_3['numbers']
# del test_4['numbers']
# del test_5['numbers']
# training_Data = []
# training_Data.append(train_1)
# training_Data.append(train_2)
# training_Data.append(train_3)
# training_Data.append(train_4)
# training_Data.append(train_5)
# testing_Data = []
# testing_Data.append(test_1)
# testing_Data.append(test_2)
# testing_Data.append(test_3)
# testing_Data.append(test_4)
# testing_Data.append(test_5)
# training_Data[0]

### Library

In [4]:
from sklearn.feature_selection import mutual_info_classif, VarianceThreshold
from sklearn.feature_selection import chi2, f_classif
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import SelectKBest
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score, roc_curve, auc, f1_score, confusion_matrix, ConfusionMatrixDisplay, accuracy_score
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, MinMaxScaler
import matplotlib.pyplot as plt
from ReliefF import ReliefF
from sklearn.linear_model import LassoCV, RidgeCV, Ridge
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

### Classifier

In [5]:
# svm = SVC(kernel='rbf', probability=True, decision_function_shape='ovr', random_state=42) ## classifier
knn = KNeighborsClassifier()
# svm = LinearSVC(multi_class='crammer_singer')

### Datasets tags

In [6]:
DATASETS = [
#     'Landsat',
#     'soybean',
#     'waveform',
#     'Splice',
#     'urban_land_cover','SCADI', 
#     'lung_discrete',
#     'CNAE-9',
#     'oh15','lung',
#     'TOX_171',
#     'orlraws10P',
#     'CLL_SUB_111',
    'CARCINOM',
    'GLIOMA',
    'AllBooks_baseline_DTM_Labelled'
]
    
DATASETS_CSV = [
#     '2_Landsat',
#     '3_soybean', 
#     '4_waveform',
#     '5_Splice',
#     '6_urban_land_cover',
#     '7_SCADI',
#     '9_lung_discrete',
#     '11_CNAE-9',
#     '12_oh15',
#     '13_lung',
#     '14_TOX_171',
#     '15_orlraws10P',
#     '16_CLL_SUB_111',
    'CARCINOM',
    'GLIOMA',
    'AllBooks_baseline_DTM_Labelled'
]

### Feature Selection Algos

In [7]:
def fs_embedded(train_data, test_data, data_temp, embedded_algo):
    
    data_ova = data_temp.copy()
    data_forXGBoost = data_temp.copy()
    del data_ova['class']
        
    data_ova = data_temp.copy()
    
    mms = pd.DataFrame(data=data_ova.drop(['class'], axis=1))
    data_mxs = MinMaxScaler().fit_transform(mms)
    data_mxs = pd.DataFrame(data=data_mxs, columns=data_ova.drop(['class'], axis=1).columns)
    data_mxs['class'] = data_ova['class']
    
#      del data_ova['class']
    del data_mxs['class']
    try:
        if(embedded_algo == 'Lasso'):
            reg = LassoCV(random_state=2)
            reg.fit(data_mxs, data_temp['class'])
        elif(embedded_algo == 'Ridge'):
            reg = RidgeCV()
            reg.fit(data_mxs, data_temp['class'])
        elif(embedded_algo == 'RandomForest'):
            reg = RandomForestClassifier(n_estimators=100, random_state=2)
            reg.fit(data_mxs, data_temp['class'])
        elif(embedded_algo == 'XGBoost'):
            reg = XGBClassifier()
            for i in range(len(data_temp)):
                data_forXGBoost['class'][i] = int(data_forXGBoost['class'][i])-1
            reg.fit(data_mxs, data_forXGBoost['class'])
            
        
        data_mxs = data_mxs.loc[:, data_mxs.columns != 'class']
        if(embedded_algo == 'RandomForest' or embedded_algo == 'XGBoost'):
            importances = reg.feature_importances_
            print(importances)
            keep_cols = [feature for feature, weight in zip (data_mxs.columns, importances) if weight != 0] 
        elif(embedded_algo == 'Lasso' or embedded_algo == 'Ridge'):
            keep_cols = [feature for feature, weight in zip (data_mxs.columns, reg.coef_) if weight != 0] 
    except:
        print("something wrong")
        keep_cols = []
    return keep_cols, len(keep_cols)

### train model

In [8]:
def train_test_clean(train_data, test_data, choosedFeature_AfterAction):
    temp = train_data.copy()
    del temp['class']
    temp_selected= temp[choosedFeature_AfterAction]
    x_train = temp_selected.sort_index(axis=1)
#     print(x_train) #80筆資料

    #把測試集手動取出與訓練集相同維度的所有資料
    temp_test = test_data.copy() #x_test
    del temp_test['class']
    temp_test_selected = temp_test[choosedFeature_AfterAction]
    x_test = temp_test_selected.sort_index(axis=1)
#     print(x_test)

    y_train = train_data['class']
    y_test = test_data['class']
    
    return x_train, x_test, y_train, y_test
def train_model(x_train, x_test, y_train, y_test, classifier):
    ## train model
#     print(x_train)
    classifier.fit(x_train, y_train)
    predicted = classifier.predict(x_test) # 預測的答案
    
#     print(predicted)
    '''-------------'''
#     predict_proba = classifier.predict_proba(x_test)
#     predict_proba = OneHotEncoder().fit_transform(predicted.reshape(-1,1)).toarray()
#     predict_proba = list(predict_proba)

#     not_predicted = []
#     for i in range(len(arange(1,7))):
#         if(i+1 not in predicted):
#             not_predicted.append(i+1)
# #     print(not_predicted)
#     if(not_predicted):
#         for i in range(len(not_predicted)):
#             for j in range(len(predict_proba)):
#                 predict_proba[j] = insert(predict_proba[j], not_predicted[i]-1, 0.)
#     print(predict_proba)
    '''-------------'''

    '''-------------'''
    lb = LabelBinarizer().fit(y_test)
    y_test_ = lb.transform(y_test)
    predict_proba = lb.transform(predicted)
    '''-------------'''

    y_test = array(list(y_test))
    predicted = array(list(predicted))
    y_train = array(list(y_train))
    
    # accuracy_train = svm.score(x_train, y_train) #x_train, y_train
    # print(accuracy_train)
#     rocaucscore = roc_auc_score(y_test_, predict_proba, multi_class='ovo',labels=arange(1,7), average='macro')
    rocaucscore = roc_auc_score(y_test_, predict_proba, average='macro')

    
    acc_scikit = accuracy_score(y_test, predicted)
#     accuracy_test = classifier.score(x_test, y_test) #x_test, y_test
    f1score = f1_score(y_test, predicted, average='macro')

    return acc_scikit, rocaucscore, f1score

def train_test_baseline(train_data, test_data):
    temp = train_data.copy()
    del temp['class']
    x_train = temp
    
    temp_test = test_data.copy()
    del temp_test['class']
    x_test = temp_test
    
    y_train = train_data['class']
    y_test = test_data['class']
    
    return x_train, x_test, y_train, y_test

### 以下方式為跑結果，分為統整跑(一次跑全部)跟個別跑(跑單一資料集的單一演算法)

### 1. 群體測試

In [ ]:
classifier = svm
# classifier = knn


# WRAPPER_ALGO = ['GA', 'PSO', 'WOA', 'GWO', 'HHO']
EMBEDDED_ALGO = ['Lasso', 'Ridge', 'RandomForest', 'XGBoost']

for z in range(len(EMBEDDED_ALGO)):
    print(f'------------------{EMBEDDED_ALGO[z]}-----------------')
    rocauc_container = []
    f1score_container = []
    acc_scikit_con = []
    SUM_OF_COLUMNS = 0
    len_of_columns = 0
    for i in range(5):     
        '''Wrapper!!!!'''
        AfterFeatureSelection, len_of_columns = fs_embedded(training_Data[i], testing_Data[i], training_Data[i], EMBEDDED_ALGO[z])
        print(AfterFeatureSelection)
        x_train, x_test, y_train, y_test = train_test_clean(training_Data[i], testing_Data[i], AfterFeatureSelection)
        SUM_OF_COLUMNS = SUM_OF_COLUMNS + len_of_columns
        acc_scikit, rocaucs, f1score = train_model(x_train, x_test, y_train, y_test, classifier)
        acc_scikit_con.append(acc_scikit)
    #     accuracy_container.append(accuracy)
        rocauc_container.append(rocaucs)
        f1score_container.append(f1score)
        print(f'accuracy of fold {i+1}: {acc_scikit}')
        print(f'rocaucs of fold {i+1}: {rocaucs}')
    
    avg_acc_scikit = sum(acc_scikit_con)/5
    # avg_acc = sum(accuracy_container)/5
    avg_rocauc = sum(rocauc_container)/5
    avg_f1score = sum(f1score_container)/5
    print('--------Baseline2 Wrapper--------')
    # print(f'avg_acc: {avg_acc}')
    print(f'avg_acc_scikit: {avg_acc_scikit}')
    print(f'avg_rocaucscore: {avg_rocauc}')
    print(f'avg_f1score: {avg_f1score}')
    print(f'avg_len: {SUM_OF_COLUMNS/5}')

### 2. 統整跑需要先做讀寫檔案操作

In [9]:
def writeFiles_Decom_UorI(STORE_PATH, DECOMPOSITION_TYPE, UorI, foldTurn, choosedFeature, f_len,FS_METHOD):
    try:
        file_U = open(STORE_PATH + f'/{DECOMPOSITION_TYPE}/{UorI}/{FS_METHOD}_folds{foldTurn+1}.txt','w')
    except:
        os.makedirs(STORE_PATH + f'/{DECOMPOSITION_TYPE}/{UorI}')
        file_U = open(STORE_PATH + f'/{DECOMPOSITION_TYPE}/{UorI}/{FS_METHOD}_folds{foldTurn+1}.txt','w')
    choosedFeature = sorted(choosedFeature)
    for item in choosedFeature:
        file_U.write(item+"\n")
    file_U.write(str(f_len)+"\n")
    file_U.close()

In [10]:
def readLines_UorI(STORE_PATH, DECOMPOSITION_TYPE, UorI, foldTurn,FS_METHOD):
    try:
        file1 = open(STORE_PATH + f'{DECOMPOSITION_TYPE}/{UorI}/{FS_METHOD}_folds{foldTurn+1}.txt', 'r')
        Lines = file1.readlines()
        choosedFeature = []
        for line in Lines:
            choosedFeature.append(line.strip())
        choosedFeature = choosedFeature[:-1]
        fs_len = Lines[-1]
        file1.close()
        return choosedFeature, fs_len
    except:
        return [], 0

In [11]:
result_baseline = pd.DataFrame(columns=['datasetName', 'algoName','acc_u','auc_u','f1_u', 'len', 'spend_time','reductionRate'])

### Embedded 統整跑

In [12]:
classifier = knn
CLASSIFIER_STRING = "KNN"
EMBEDDED_ALGO = ['Lasso', 'Ridge', 'RandomForest', 'XGBoost']

dir_data = '../datasets/'
for qq in range(len(DATASETS)):
    DATASET_CSV_NAME = f'{DATASETS_CSV[qq]}.csv'
    DATASET_NAME = f'{DATASETS[qq]}'
    dir_train_test_data = f'../train_test_datasets/{DATASET_NAME}'

    train_data_loc = os.path.join(dir_data, DATASET_CSV_NAME)
    print('Path of read in data: %s' % (train_data_loc))
    data = pd.read_csv(train_data_loc)
    # data
    
    CLASS_AMOUNT = data['class'].value_counts().size
    FEATURE_AMOUNT = len(data.columns)-2
    # CLASS_AMOUNT

    train_1_data_loc = os.path.join(dir_train_test_data, 'train_1.csv')
    train_1 = pd.read_csv(train_1_data_loc)
    train_2_data_loc = os.path.join(dir_train_test_data, 'train_2.csv')
    train_2 = pd.read_csv(train_2_data_loc)
    train_3_data_loc = os.path.join(dir_train_test_data, 'train_3.csv')
    train_3 = pd.read_csv(train_3_data_loc)
    train_4_data_loc = os.path.join(dir_train_test_data, 'train_4.csv')
    train_4 = pd.read_csv(train_4_data_loc)
    train_5_data_loc = os.path.join(dir_train_test_data, 'train_5.csv')
    train_5 = pd.read_csv(train_5_data_loc)
    test_1_data_loc = os.path.join(dir_train_test_data, 'test_1.csv')
    test_1 = pd.read_csv(test_1_data_loc)
    test_2_data_loc = os.path.join(dir_train_test_data, 'test_2.csv')
    test_2 = pd.read_csv(test_2_data_loc)
    test_3_data_loc = os.path.join(dir_train_test_data, 'test_3.csv')
    test_3 = pd.read_csv(test_3_data_loc)
    test_4_data_loc = os.path.join(dir_train_test_data, 'test_4.csv')
    test_4 = pd.read_csv(test_4_data_loc)
    test_5_data_loc = os.path.join(dir_train_test_data, 'test_5.csv')
    test_5 = pd.read_csv(test_5_data_loc)
    del train_1['numbers']
    del train_2['numbers']
    del train_3['numbers']
    del train_4['numbers']
    del train_5['numbers']
    del test_1['numbers']
    del test_2['numbers']
    del test_3['numbers']
    del test_4['numbers']
    del test_5['numbers']
    training_Data = []
    training_Data.append(train_1)
    training_Data.append(train_2)
    training_Data.append(train_3)
    training_Data.append(train_4)
    training_Data.append(train_5)
    testing_Data = []
    testing_Data.append(test_1)
    testing_Data.append(test_2)
    testing_Data.append(test_3)
    testing_Data.append(test_4)
    testing_Data.append(test_5)
    
    spend_time = 0
    for z in range(len(EMBEDDED_ALGO)):
        print(f'------------------{EMBEDDED_ALGO[z]}-----------------')
        STORE_PATH = f'./EMBEDDED_{EMBEDDED_ALGO[z]}/{DATASET_NAME}/'
        for i in range(5):     
            start_time = time.time()
            AfterFeatureSelection, len_of_columns = fs_embedded(training_Data[i], testing_Data[i], training_Data[i], EMBEDDED_ALGO[z])
    #         print(AfterFeatureSelection)
            end_time = time.time()
            spend_time += end_time - start_time
            writeFiles_Decom_UorI(STORE_PATH, 'BASELINE', 'N', i, AfterFeatureSelection, len_of_columns, EMBEDDED_ALGO[z])
        spend_time = spend_time//5
    
    for kk in range(len(EMBEDDED_ALGO)):
        STORE_PATH = f'./EMBEDDED_{EMBEDDED_ALGO[kk]}/{DATASET_NAME}/'
        rocauc_container = []
        f1score_container = []
        acc_scikit_con = []
        SUM_OF_COLUMNS = 0
        len_of_columns = 0
        for i in range(5):
            AfterFeatureSelection, len_of_columns = readLines_UorI(STORE_PATH, 'BASELINE', 'N', i, EMBEDDED_ALGO[kk])
            if(AfterFeatureSelection  != []):
                x_train, x_test, y_train, y_test = train_test_clean(training_Data[i], testing_Data[i], AfterFeatureSelection)
                SUM_OF_COLUMNS = SUM_OF_COLUMNS + int(len_of_columns)
                acc_scikit, rocaucs, f1score = train_model(x_train, x_test, y_train, y_test, classifier)
                acc_scikit_con.append(acc_scikit)
        #     accuracy_container.append(accuracy)
                rocauc_container.append(rocaucs)
                f1score_container.append(f1score)
            else:
                acc_scikit_con.append(0)
                rocauc_container.append(0)
                f1score_container.append(0)
            print(f'accuracy of fold {i+1}: {acc_scikit}')
            print(f'rocaucs of fold {i+1}: {rocaucs}')
        
        avg_acc_scikit = sum(acc_scikit_con)/5
        # avg_acc = sum(accuracy_container)/5
        avg_rocauc = sum(rocauc_container)/5
        avg_f1score = sum(f1score_container)/5
        print('--------Baseline2 Embedded--------')
        # print(f'avg_acc: {avg_acc}')
        print(f'avg_acc_scikit: {avg_acc_scikit}')
        print(f'avg_rocaucscore: {avg_rocauc}')
        print(f'avg_f1score: {avg_f1score}')
        print(f'avg_len: {SUM_OF_COLUMNS/5}')
    
        t = pd.DataFrame([{'datasetName':DATASET_NAME,
                     'algoName':EMBEDDED_ALGO[z],
                     'acc_u':str(round(avg_acc_scikit,3)),
                     'auc_u':str(round(avg_rocauc,3)),
                     'f1_u':str(round(avg_f1score,3)),
                     'len':str(round(SUM_OF_COLUMNS/5,3)),
                     'spend_time':str(round(spend_time,3)),
                     'reductionRate':str(round((SUM_OF_COLUMNS/5)/FEATURE_AMOUNT, 3))
                    }])
        result_baseline = pd.concat([result_baseline, t]).reset_index(drop=True).astype('string')

    
with pd.ExcelWriter(f'./result_excel_embedded/result_baseline_fs_embedded_{CLASSIFIER_STRING}_230917.xlsx') as writer:
    result_baseline.to_excel(writer, sheet_name='baseline_svm_fs')

Path of read in data: ../datasets/CARCINOM.csv
------------------Lasso-----------------
------------------Ridge-----------------
------------------RandomForest-----------------
[0.         0.         0.         ... 0.00031917 0.         0.        ]
[0. 0. 0. ... 0. 0. 0.]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
 8.11030008e-05 0.00000000e+00]
[0.         0.00070144 0.00022391 ... 0.         0.00010814 0.        ]
[0.         0.         0.         ... 0.00134471 0.         0.        ]
------------------XGBoost-----------------
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0.00329664 0.         0.         ... 0.         0.         0.        ]
[0.00170978 0.         0.         ... 0.         0.         0.        ]
accuracy of fold 1: 0.7631578947368421
rocaucs of fold 1: 0.8384771825396825
accuracy of fold 2: 0.9117647058823529
rocaucs of fold 2: 0.8893841431556949
accuracy of fold 3: 0.8235294117647058
rocaucs of fold 3: 0.8528637669801

accuracy of fold 1: 0.5901639344262295
rocaucs of fold 1: 0.6610194757499785
accuracy of fold 2: 0.6581196581196581
rocaucs of fold 2: 0.787249627162141
accuracy of fold 3: 0.6239316239316239
rocaucs of fold 3: 0.6808612721302076
accuracy of fold 4: 0.5555555555555556
rocaucs of fold 4: 0.710285971885654
accuracy of fold 5: 0.5470085470085471
rocaucs of fold 5: 0.729408899457349
--------Baseline2 Embedded--------
avg_acc_scikit: 0.5949558638083229
avg_rocaucscore: 0.713765049277066
avg_f1score: 0.4963688843342277
avg_len: 289.0


In [ ]:
classifier = knn
CLASSIFIER_STRING = "KNN"
EMBEDDED_ALGO = ['Lasso', 'Ridge', 'RandomForest', 'XGBoost']

dir_data = '../datasets/'
for qq in range(len(DATASETS)):
    DATASET_CSV_NAME = f'{DATASETS_CSV[qq]}.csv'
    DATASET_NAME = f'{DATASETS[qq]}'
    dir_train_test_data = f'../train_test_datasets/{DATASET_NAME}'

    train_data_loc = os.path.join(dir_data, DATASET_CSV_NAME)
    print('Path of read in data: %s' % (train_data_loc))
    data = pd.read_csv(train_data_loc)
    # data
    
    CLASS_AMOUNT = data['class'].value_counts().size
    FEATURE_AMOUNT = len(data.columns)-2
    # CLASS_AMOUNT

    train_1_data_loc = os.path.join(dir_train_test_data, 'train_1.csv')
    train_1 = pd.read_csv(train_1_data_loc)
    train_2_data_loc = os.path.join(dir_train_test_data, 'train_2.csv')
    train_2 = pd.read_csv(train_2_data_loc)
    train_3_data_loc = os.path.join(dir_train_test_data, 'train_3.csv')
    train_3 = pd.read_csv(train_3_data_loc)
    train_4_data_loc = os.path.join(dir_train_test_data, 'train_4.csv')
    train_4 = pd.read_csv(train_4_data_loc)
    train_5_data_loc = os.path.join(dir_train_test_data, 'train_5.csv')
    train_5 = pd.read_csv(train_5_data_loc)
    test_1_data_loc = os.path.join(dir_train_test_data, 'test_1.csv')
    test_1 = pd.read_csv(test_1_data_loc)
    test_2_data_loc = os.path.join(dir_train_test_data, 'test_2.csv')
    test_2 = pd.read_csv(test_2_data_loc)
    test_3_data_loc = os.path.join(dir_train_test_data, 'test_3.csv')
    test_3 = pd.read_csv(test_3_data_loc)
    test_4_data_loc = os.path.join(dir_train_test_data, 'test_4.csv')
    test_4 = pd.read_csv(test_4_data_loc)
    test_5_data_loc = os.path.join(dir_train_test_data, 'test_5.csv')
    test_5 = pd.read_csv(test_5_data_loc)
    del train_1['numbers']
    del train_2['numbers']
    del train_3['numbers']
    del train_4['numbers']
    del train_5['numbers']
    del test_1['numbers']
    del test_2['numbers']
    del test_3['numbers']
    del test_4['numbers']
    del test_5['numbers']
    training_Data = []
    training_Data.append(train_1)
    training_Data.append(train_2)
    training_Data.append(train_3)
    training_Data.append(train_4)
    training_Data.append(train_5)
    testing_Data = []
    testing_Data.append(test_1)
    testing_Data.append(test_2)
    testing_Data.append(test_3)
    testing_Data.append(test_4)
    testing_Data.append(test_5)
    
    for kk in range(len(EMBEDDED_ALGO)):
        STORE_PATH = f'./EMBEDDED_{EMBEDDED_ALGO[kk]}/{DATASET_NAME}/'
        rocauc_container = []
        f1score_container = []
        acc_scikit_con = []
        SUM_OF_COLUMNS = 0
        len_of_columns = 0
        for i in range(5):
            AfterFeatureSelection, len_of_columns = readLines_UorI(STORE_PATH, 'BASELINE', 'N', i, EMBEDDED_ALGO[kk])
            if(AfterFeatureSelection  != []):
                x_train, x_test, y_train, y_test = train_test_clean(training_Data[i], testing_Data[i], AfterFeatureSelection)
                SUM_OF_COLUMNS = SUM_OF_COLUMNS + int(len_of_columns)
                acc_scikit, rocaucs, f1score = train_model(x_train, x_test, y_train, y_test, classifier)
                acc_scikit_con.append(acc_scikit)
        #     accuracy_container.append(accuracy)
                rocauc_container.append(rocaucs)
                f1score_container.append(f1score)
            else:
                acc_scikit_con.append(0)
                rocauc_container.append(0)
                f1score_container.append(0)
            print(f'accuracy of fold {i+1}: {acc_scikit}')
            print(f'rocaucs of fold {i+1}: {rocaucs}')
        
        avg_acc_scikit = sum(acc_scikit_con)/5
        # avg_acc = sum(accuracy_container)/5
        avg_rocauc = sum(rocauc_container)/5
        avg_f1score = sum(f1score_container)/5
        print('--------Baseline2 Embedded--------')
        # print(f'avg_acc: {avg_acc}')
        print(f'avg_acc_scikit: {avg_acc_scikit}')
        print(f'avg_rocaucscore: {avg_rocauc}')
        print(f'avg_f1score: {avg_f1score}')
        print(f'avg_len: {SUM_OF_COLUMNS/5}')
    
        t = pd.DataFrame([{'datasetName':DATASET_NAME,
                     'algoName':EMBEDDED_ALGO[z],
                     'acc_u':str(round(avg_acc_scikit,3)),
                     'auc_u':str(round(avg_rocauc,3)),
                     'f1_u':str(round(avg_f1score,3)),
                     'len':str(round(SUM_OF_COLUMNS/5,3)),
                     'spend_time':str(round(spend_time,3)),
                     'reductionRate':str(round((SUM_OF_COLUMNS/5)/FEATURE_AMOUNT, 3))
                    }])
        result_baseline = pd.concat([result_baseline, t]).reset_index(drop=True).astype('string')
    

In [19]:
with pd.ExcelWriter(f'./result_excel_embedded/result_baseline_fs_embedded.xlsx') as writer:
    result_baseline.to_excel(writer, sheet_name='baseline_svm_fs')